In [ ]:
import requests
import json
from urllib.parse import urljoin

def fetch_official_reseed_urls():
    """Fetch current reseed URLs from I2P official sources"""
    official_sources = [
        "https://geti2p.net/en/get-involved/guides/reseed",
        "https://github.com/i2p/i2p.i2p/blob/master/installer/resources/reseed_urls.txt"
    ]

    reseed_urls = [
        "https://reseed.i2p-projekt.de/",
        "https://i2p.mooo.com/netDb/",
        "https://netdb.i2p2.no/",
        "https://us.reseed.i2p2.no:444/",
        "https://uk.reseed.i2p2.no:444/",
        "https://i2p-0.manas.ca:8443/",
        "https://reseed.i2p.vzaws.com:8443/",
        "https://user.mx24.eu/",
        "https://download.xxlspeed.com/",
        "https://reseed-fr.i2pd.xyz:8443/",
        "https://reseed.onion.im/",
        "https://i2pseed.zarrenspry.info/",
        "https://reseed.diva.exchange/",
        "https://i2pseeds.spongebobminer.com/",
        "https://reseed.atomike.ninja/"
    ]

    print(f"Found {len(reseed_urls)} official reseed URLs")
    return reseed_urls

In [ ]:
import ssl
import socket
from urllib.request import urlopen, Request
from urllib.error import URLError, HTTPError
import time

def download_reseed_data(reseed_url, timeout=10):
    """Download reseed data manually from working servers"""
    try:
        # Create SSL context that accepts self-signed certificates
        ssl_context = ssl.create_default_context()
        ssl_context.check_hostname = False
        ssl_context.verify_mode = ssl.CERT_NONE

        # Prepare request with proper headers
        headers = {
            'User-Agent': 'I2P Bootstrap Client 1.0',
            'Accept': 'application/x-i2p-netdb',
            'Connection': 'close'
        }

        request = Request(reseed_url, headers=headers)

        print(f"Attempting to download from: {reseed_url}")

        with urlopen(request, timeout=timeout, context=ssl_context) as response:
            if response.status == 200:
                data = response.read()
                print(f"✓ Successfully downloaded {len(data)} bytes from {reseed_url}")
                return data
            else:
                print(f"✗ HTTP {response.status} from {reseed_url}")
                return None

    except (URLError, HTTPError, socket.timeout) as e:
        print(f"✗ Failed to download from {reseed_url}: {e}")
        return None
    except Exception as e:
        print(f"✗ Unexpected error with {reseed_url}: {e}")
        return

In [ ]:
import concurrent.futures
from threading import Lock

def verify_reseed_urls(reseed_urls, max_workers=5, timeout=10):
    """Verify reseed URLs and build a dynamic list of working servers"""
    working_urls = []
    failed_urls = []
    results_lock = Lock()

    def test_single_url(url):
        try:
            # Test connectivity first
            ssl_context = ssl.create_default_context()
            ssl_context.check_hostname = False
            ssl_context.verify_mode = ssl.CERT_NONE

            request = Request(url, headers={'User-Agent': 'I2P Bootstrap Test'})

            with urlopen(request, timeout=timeout, context=ssl_context) as response:
                status = response.status
                content_length = response.headers.get('Content-Length', 'Unknown')

                with results_lock:
                    if status == 200:
                        working_urls.append({
                            'url': url,
                            'status': status,
                            'content_length': content_length,
                            'response_time': time.time()
                        })
                        print(f"✓ {url} - Status: {status}, Size: {content_length}")
                    else:
                        failed_urls.append({'url': url, 'status': status, 'error': f'HTTP {status}'})
                        print(f"✗ {url} - Status: {status}")

        except Exception as e:
            with results_lock:
                failed_urls.append({'url': url, 'status': None, 'error': str(e)})
                print(f"✗ {url} - Error: {e}")

    print(f"Testing {len(reseed_urls)} reseed URLs with {max_workers} workers...")

    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        executor.map(test_single_url, reseed_urls)

    # Sort working URLs by response time
    working_urls.sort(key=lambda x: x.get('response_time', float('inf')))

    print(f"\n=